In [1]:
# criar o navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time

# Configurando o WebDriver com a versão mais recente
servico = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome(service=servico)




In [2]:
# abrir a página index (entrar no site da busca jurídica)
import os

pasta = os.getcwd()
arquivo = 'index.html'
caminho = os.path.join(pasta, arquivo)

driver.get(caminho)


In [60]:
# Interagir om botão de escolher estado para habilitar a escolha do estado
from selenium.webdriver.common.action_chains import ActionChains

mouse = ActionChains(driver)

botao_estado = driver.find_element(By.CLASS_NAME, 'dropdown-menu')

mouse.move_to_element(botao_estado).perform()

driver.find_element(By.XPATH, '/html/body/div/div/div/a[2]').click()
time.sleep(0.5)
mouse.move_to_element(botao_estado).perform()
driver.find_element(By.XPATH, '/html/body/div/div/div/a[1]').click()
time.sleep(0.5)
driver.find_element(By.XPATH, '/html/body/div/div/div/a[3]').click()

In [86]:
# Mudar de aba
lista_abas = driver.window_handles
print(lista_abas)

for aba in driver.window_handles:
    driver.switch_to.window(aba)
    print(driver.title)

aba_original = lista_abas[0]
aba_rj = lista_abas[1]
aba_df = lista_abas[2]
aba_sp = lista_abas[3]

driver.switch_to.window(aba_rj)
driver.find_element(By.ID, 'nome').send_keys('Adamastor Pitaco')
driver.find_element(By.ID, 'advogado').send_keys('Professor Raymundo')
driver.find_element(By.ID,'numero').send_keys('123123')
driver.find_element(By.CLASS_NAME,'registerbtn').click()



['DDB5805B685C4FA93E4F565ED77962F3', '7EE5915F81E9D1D2E03D2CBA776D42DB', '3A458047FC9786AAD7445C061FE1EA82', '5FA61A6AF422FFE95896F60DCF7B336A', 'A678A43D1806349359B63FB1AC5073EC']




DevTools


In [87]:
# Mudando pra alerta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert

alerta = driver.switch_to.alert
alerta.accept()

espera = WebDriverWait(driver, 20)
alerta = espera.until(EC.alert_is_present())
texto_alerta = alerta.text.lower()
alerta.accept()
time.sleep(2)
if 'processo encontrado com' in texto_alerta:
    print('Processo encontraado')
else:
    print('Processo não encontrado')
driver.refresh() #recarregar a página
    




Processo não encontrado


In [3]:
# importar a base de dados
import pandas as pd

tabela = pd.read_excel('Processos.xlsx')
display(tabela)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,NaN
1,João,Lawyer Alon,EB3792,Rio de Janeiro,NaN
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,NaN
3,Carol,Amanda,PC5197,São Paulo,NaN


In [4]:
# para cada processo (linha da tabela)
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
for linha in tabela.index:
    
   
    
    cidade = tabela.loc[linha, 'Cidade']    

    # abrir a lista de cidades
    mouse = ActionChains(driver)
    botao = driver.find_element(By.CLASS_NAME,'dropdown-menu')
    mouse.move_to_element(botao).perform()
    
    
    # selecionando o cidade
    driver.find_element(By.PARTIAL_LINK_TEXT, cidade).click()
    # mudar para a nova aba
    aba_original = driver.window_handles[0]
    #indice = 1 + linha
    aba_nova = driver.window_handles[1]
    driver.switch_to.window(aba_nova)
        
    # preencher o formulário com os dados de busca
    driver.find_element(By.ID, 'nome').send_keys(tabela.loc[linha, 'Nome'])
    driver.find_element(By.ID, 'advogado').send_keys(tabela.loc[linha, 'Advogado'])
    driver.find_element(By.ID, 'numero').send_keys(tabela.loc[linha, 'Processo'])
    

    # clicar em pesquisar
    driver.find_element(By.CLASS_NAME,'registerbtn').click()
   
    
    # confirmar a pesquisa
    alerta = driver.switch_to.alert
    alerta.accept()
   
    # esperar o resultado da pesquisa e agir de acordo com o resultado
    espera = WebDriverWait(driver, 15)
    alerta = espera.until(EC.alert_is_present())
    texto_alerta = alerta.text
    if 'Deseja' in texto_alerta:
        status = 'Encontrado'
    else:
        status = 'Não encontrado'
    tabela.loc[linha, 'Status'] = status
    alerta.accept()
    time.sleep(1)
    driver.close()
    driver.switch_to.window(aba_original)
    

   

C:\Users\Dell\AppData\Local\Temp\ipykernel_19936\3954406670.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Encontrado' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tabela.loc[linha, 'Status'] = status


In [5]:
driver.quit()
display(tabela)

tabela.to_excel("Processos Atualizado.xlsx")

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,Encontrado
1,João,Lawyer Alon,EB3792,Rio de Janeiro,Não encontrado
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,Não encontrado
3,Carol,Amanda,PC5197,São Paulo,Encontrado
